<a href="https://colab.research.google.com/github/JulianaCarvajal/Spaceship_Titanic/blob/workOnPreprocess%2FBustamJos3/dataset_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#libraries
from zipfile import ZipFile
import pandas as pd

# Working with the next cell [Online]
For now on, import cell from colab will have this code (**On local notebook, respective changes must be made in order to import .json token and datasets from kaggle**).

In [2]:
#call API
%pip install kaggle
#upload kaggle token .json
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"bustamjos3","key":"a80d020fd1917115be9ce5ce749b71d9"}'}

In [3]:
#Make a directory named kaggle and copy the kaggle.json file there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# change the permission of the file
!chmod 600 ~/.kaggle/kaggle.json
# Import data files from the Kaggle API
!kaggle competitions download -c spaceship-titanic
#unzip files
file_name = "spaceship-titanic.zip" #the file is the dataset exact name
with ZipFile("spaceship-titanic.zip", 'r') as zip:
  zip.extractall()
  print('Done')
# .zip is now not necessary
!rm *.zip

  0% 0.00/299k [00:00<?, ?B/s]
100% 299k/299k [00:00<00:00, 69.4MB/s]
Done


# End of working [Online]

# Working on Local

In [ ]:
#call API
%pip install kaggle
# Import data files from the Kaggle API
!kaggle competitions download -c spaceship-titanic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Unzip the data files
with ZipFile("spaceship-titanic.zip", 'r') as zip_ref:
    zip_ref.extractall()
    print('Done')
# .zip is now not necessary
!rm *.zip

# End of working on Local

In [40]:
# Read the data files into pandas dataframes
d_sTitanic = pd.read_csv("train.csv")
d_sTitanic.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


# Pipeline for work on preprocessing

## Drop at 1st looking, not relevant columns:
* ```d_sTitanic['PassengerId']```
* ```d_sTitanic['Name']```

In [41]:
#drop those columns
d_sTitanic.drop(['PassengerId','Name'],axis=1,inplace=True)

In [26]:
# separation of X, y
y=d_sTitanic.iloc[:,-1].values

In [102]:
#also drop
d_sTitanic.drop(['Transported'],axis=1,inplace=True)

In [103]:
# take name cols which are categorical
listCategoric=[str(i) for i in (d_sTitanic.dtypes=='O').loc[(d_sTitanic.dtypes=='O')==True].index]

In [104]:
# take the categoric types only
XCategoric=d_sTitanic[listCategoric].values

In [105]:
#numeric col names
listNumeric=[str(i) for i in (d_sTitanic.dtypes=='O').loc[(d_sTitanic.dtypes=='float64')==True].index]

In [106]:
# take the numeric types only
XNumeric=d_sTitanic[listNumeric].values

## oneHotEncoding
Necessary to be done on previous 'cause working with the next methods require array-like data

In [30]:
from sklearn.preprocessing import OneHotEncoder

In [31]:
# instancing
oneHotEncoder=OneHotEncoder()

In [107]:
#fitting with categorical X
oneHotEncoder.fit_transform(XCategoric).toarray()

array([[0., 1., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       ...,
       [1., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.]])

## ```NaN``` Imputation

In [21]:
import numpy as np
from sklearn.impute import KNNImputer

In [22]:
#instancing
kNNImputer=KNNImputer(n_neighbors=1,weights='distance',copy=True)

In [29]:
kNNImputer.fit_transform(X)

ValueError: ignored

In [ ]:
# checking variances in order to take a 1st look of candidates to be dropped, categorical types ignored
d_sTitanic.var(numeric_only=True).sort_values(ascending=False)

FoodCourt       2.596898e+06
VRDeck          1.312668e+06
Spa             1.292099e+06
RoomService     4.445124e+05
ShoppingMall    3.656578e+05
Age             2.099317e+02
Transported     2.500156e-01
dtype: float64

In [ ]:
# import train_test split function
from sklearn.model_selection import train_test_split

In [ ]:
# train validation split
X_trainST,X_valST,y_trainST,y_valST=train_test_split(X,y,test_size=0.33,random_state=42)

In [ ]:
# standardization is required 'cause many models work badly is data does'nt look like normally distributed data (see biblio)
from sklearn.preprocessing import StandardScaler

## Necessary
1. Drop cols with 0 importance
2. Make oneHotEncode to transform categorical to discrete. This is important 'cause preprocessing functions only work with numerical values!

In [ ]:
# scaling with train
standardS=StandardScaler().fit(X_trainST)

ValueError: ignored

# End of work on preprocessing